In [ ]:
!pip install folium
!pip install PyPDF2
!pip install urllib3
!pip install pdfminer
!pip3 install table_ocr

     |████████████████████████████████| 93 kB 2.8 MB/s eta 0:00:011
     |████████████████████████████████| 77 kB 6.1 MB/s eta 0:00:011
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-py3-none-any.whl size=61084 sha256=bcf11225beeb8f036d234b957a1d10e6534700815311b53388636ece089c4cea
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/80/1a/24/648467ade3a77ed20f35cfd2badd32134e96dd25ca811e64b3
Successfully built PyPDF2
     |████████████████████████████████| 4.2 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.7 MB 52.6 MB/s eta 0:00:01

## 1. Importing Libraries:

In [ ]:
import numpy as np # library to handle data in a vectorized manner
from pprint import pprint

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
from urllib.request import urlopen

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#Library to extract text and data from pdf format
import PyPDF2
import urllib3
from io import StringIO
import io

print('Libraries imported.')

# **1. Introduction**

The first human cases of COVID-19 were identified in Wuhan, Hubei, China, in December 2019. As a result of COVID-19 many regions have imposed lockdowns, curfews and quarantines alongside new legislation and evacuations, or other restrictions for citizens of or recent travelers to the most affected areas.[1] Other regions have imposed global restrictions that apply to all foreign countries and territories, or prevent their own citizens from travelling overseas.[2]
The pandemic was confirmed to have reached South America on 26 February 2020 when Brazil confirmed a case in São Paulo.[3] By 3 April, all countries and territories in South America had recorded at least one case.[4]
In Ecuador, the virus was reported to have spread to Ecuador on 29 February 2020 when a woman in her 70s tested positive for the virus. This disease has caused a several disrupt in the economy and life for every citizen in Ecuador. As result, this project is targeted to get more knowledge about this disease. Spefically, finding hot spots and warn people about the situation of each province in Ecuador. 

Referency entry:
1. "Coronavirus Travel Restrictions, Across the Globe". The New York Times. 26 March 2020.
2. "Coronavirus (COVID-19) – information for Australian travelers". Australian Government. Retrieved 4 September 2020.
3. Horwitz L, Nagovitch P, Sonnel HK, Zissis C. "Where Is the Coronavirus in Latin America?". AS/COA. Archived from the original on 22 March 2020. Retrieved 22 March 2020.
4. "Uncollected bodies lie for days in the streets of Ecuador the emerging epicentre of the coronavirus in Latin America". Stuff/Fairfax. 4 April 2020.

## **2. Description of the problem**


"*Ecuador has one of the highest per capita Covid-19 deaths rates in the world, according to research from Johns Hopkins University. A New York Times analysis of over two dozen countries found that Ecuador had one of the highest excess death rate* "[1] . Therefore, as a current resident of Ecuador, I am concerned about my family safety in this country ,and I want to know where are located the highest spot of Covid in this country. This information will help helpful to travel within this country and acknowledge Ecuadorians where they Ecuador can stay safely from this brutal disease. 

**Reference entry:**
1. url: Noah Smith, [Covid-19 in Ecuador: An Exclusive Q&A with Ecuador’s Ambassador to the U.S.](https://reliefweb.int/report/ecuador/covid-19-ecuador-exclusive-qa-ecuador-s-ambassador-us)


## **2.1. Background**

**Country-Specific Information:**

1.   Ecuador has confirmed positive cases of COVID-19 in all 24 provinces.
2.   On August 6, the U.S. Department of State removed the Global Health Advisory Level 4. Ecuador is now listed as Level 3: Reconsider Travel[1].
3.   The COVID-19 pandemic in Ecuador is part of the worldwide pandemic of coronavirus disease 2019 (COVID-19) caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). The virus was reported to have spread to Ecuador on 29 February 2020 when a woman in her 70s tested positive for the virus. Ecuador was described in April as emerging as a possible "epicentre" of the pandemic in Latin America,[2] with the city of Guayaquil overwhelmed to the point where bodies were being left in the street[3].


**Reference entry:**
1. url: CS Unit, 19 October, 2020 url: https://ec.usembassy.gov/covid-19-information-ecu-2/
2."Uncollected bodies lie for days in the streets of Ecuador the emerging epicentre of the coronavirus in Latin America". Stuff/Fairfax. 4 April 2020.
3. "Ecuador coronavirus: Bodies are being left in the streets in an overwhelmed city". msn.com. Retrieved 5 April 2020.




### **3. Description of Data:**
I am usig several input of data:

1.   The first input of data that I am using is from wikipedia. Specifically, it is a page with data about COVID-19 pandemic in Ecuador.

  URL: "https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Ecuador"

    

*   This URL has information about how many confirmed cases and deaths are in 19_pandemic_in_Ecuador Ecuador. This data is classified by each province in Ecuador.  

2.   The second input of data are the geolocation data of each provinces in Ecuador. This information was extracted using geopy locator and can provide the latitude and longitude of each province in Ecuador.

3. The final data are the nearby venues in Ecuador that were extracted using the API foursquare. This data will help to correlate a cluster Covid hotspots in Ecuador. 




### **4. Methodology**

There are several methodology implemented within this project. The following include:

*   Web scrapping.
*   Organization an collection of data.
*   Geolocation of ach province and nearby venues.
*   Plotting with folium. 



### ***5. Results section***

### Using web scrapping and beautiful soup to extract data from a wikipedia page.

In [ ]:
wiki_URL="https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Ecuador"

In [ ]:
#from inspecting element you get the class wikitable sortable jquery-tablesorter
response=requests.get(wiki_URL)

In [ ]:
soup = BeautifulSoup(response.text,'lxml')
table=soup.find('table',{'class':'wikitable'}).tbody

### First, I cleaned the HTML headers by converting it to text and using remove_html tags function.


In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile(r'(<[^\>]*>)|(\n)|(\xa0)') 
    return re.sub(clean, '', text) 

In [ ]:
header=[]
for th in table.find_all('th'):
  header.append(str(th))
header=list(map(remove_html_tags,header[0:4]))
pprint(header)

In [ ]:
rows = []
for td in table.find_all('td'):
    rows.append(str(td))
rows=list(map(remove_html_tags,rows))
print(rows, sep=",")

After cleaning our colums and rows, I am going to construct the dataframe. 

1. First, I construct a list with the provinces  with its data  where each province has it owns data contained in a list. 

In [ ]:
dict_kv=[]
for i in range(0, len(rows)):
  if (rows[i].strip().isalpha()):  
    dict_kv.append([rows[i],rows[i+1],rows[i+2],rows[i+3]])
    
  
dict_kv.append([rows[92],rows[93],rows[94],rows[95]])  # I need to accomodate the last item of the list to get the complete items
      
pprint(dict_kv)
type(dict_kv)

In [ ]:
df = pd.DataFrame(data=dict_kv, columns = header)
df

#### Then, I want to get the coordinates of Ecuador and its provices.

In [ ]:
address = 'Ecuador'

geolocator = Nominatim(user_agent="ecuador_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the country are {}, {}. This country is {}.'.format(latitude, longitude, location))

Create a map of Ecuador

In [ ]:
# create map of Toronto using latitude and longitude values
map_ecuador = folium.Map(location=[latitude, longitude], tiles='OpenStreetMap', zoom_start=12)
map_ecuador

Then, I want to obtain the latitude and longitude for each provice in ecuador

In [ ]:
address_list=[]
latitude=[]
longitude=[]

for x in df['Provinces']:
  address=str(x)+",Ecuador"
  geolocator = Nominatim(user_agent="ecuador_explorer")
  location = geolocator.geocode(address)
  latitude.append(location.latitude)
  longitude.append(location.longitude)
  
df2= pd.DataFrame({'Latitude':latitude,'Longitude':longitude})
df2.head()


Then, I joined the two dataframes and show the results.


In [ ]:
ecuador = df.combine_first(df2)
ecuador.sort_values(['Provinces'], inplace=True)
ecuador

The Foursquare API is going to be used to explore the neighborhoods.

In [ ]:
CLIENT_ID = 'CB245Q52IIDUCQ1VXR2HBN5HANNGGSNTWW2CZ0K2Z4PQWAZT' # your Foursquare ID
CLIENT_SECRET = 'M4O5SZNTHG2JBVJSERHF10H2NC5IX3OMC20IIKHDLA2KHFWB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

I decided to explore Guayaquil, which is the provinces where I live.

In [ ]:
print(ecuador[ecuador['Provinces']=='Guayas'].index.values) # First, I get the index of my province.
print(ecuador.loc[8, 'Provinces'])
guayas_latitude=ecuador.loc[8,'Latitude'] #  guayas latitude value
guayas_longitude=ecuador.loc[8,'Longitude']
print("guayas latitude: {}".format(guayas_latitude)) # guayas longitude value
print("guayas longitude: {}".format(guayas_longitude)) # guayas longitude value

Now, let's get the top 100 venues that are Guayas within a radius of 1000 meters

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 100000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    guayas_latitude, 
    guayas_longitude, 
    radius, 
    LIMIT)
url # display URL

Send the GET request and examine the results

In [ ]:
ec_results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Then, we get the venues.

In [ ]:
ec_venues = ec_results['response']['groups'][0]['items']
    
ec_nearby_venues = json_normalize(ec_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
ec_nearby_venues =ec_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
ec_nearby_venues['venue.categories'] = ec_nearby_venues.apply(get_category_type, axis=1)

# clean columns
ec_nearby_venues.columns = [col.split(".")[-1] for col in ec_nearby_venues.columns]
ec_nearby_venues.head()


100 venues were returned.

In [ ]:
print('{} venues were returned by Foursquare.'.format(ec_nearby_venues.shape[0]))

Explore provinces in Ecuador

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=100000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
ecuador_venues = getNearbyVenues(names=ecuador['Provinces'],
                                   latitudes=ecuador['Latitude'],
                                   longitudes=ecuador['Longitude']
                                  )

Let's check the size of the resulting dataframe

In [ ]:
print(ecuador_venues.shape)

In [ ]:
# add markers to map
for lat, lng, provinces, confirmedCases, ConfirmedDeaths in zip(ecuador['Latitude'], ecuador['Longitude'], ecuador['Provinces'], ecuador['Confirmed cases'], ecuador['Confirmed deaths']):
    label = 'Provinces: {}, Confirmed Cases: {}, Deaths: {}'.format(provinces, confirmedCases ,ConfirmedDeaths)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=(5*float(confirmedCases.replace(',',''))/59477),
        popup=label,
        color='red',
        fill='red',
        fill_color='#FF0000',
        fill_opacity=(float(confirmedCases.replace(',',''))/59477),
        parse_html=False).add_to(map_ecuador)
  

    


In [ ]:
map_ecuador

# ***5. Discussion section***

In this section any observations and any recommendations will be discussed.

# **6. Conclusion:** 
With this information, I conclude that Pichincha is most dangerous provinces to visit. I can make more further analysis about this data. This analysis will continue on week 5.